In [ ]:
from notebookutils import mssparkutils
import datetime

path = 'Files/data/'
starts_with_total = 'data_total_'
starts_with_fact = 'fact_table_'

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 17, Finished, Available, Finished)

In [ ]:
def latest_file(path, file_name_starts_with):
    files = [item.name for item in mssparkutils.fs.ls(path) if item.name.startswith(file_name_starts_with)]

    newest_file = None
    newest_time = None

    for file in files:
        timestamp = file[11:26]
        dt = datetime.datetime.strptime(timestamp, '%Y%m%d %H%M%S')
        if newest_time is None or dt > newest_time:
            newest_time = dt
            newest_file = file

    print(newest_file)
    return newest_file

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 18, Finished, Available, Finished)

In [ ]:
latest_total_count_file = latest_file(path, starts_with_total)
latest_fact_file = latest_file(path, starts_with_fact)

total_count = spark.read.parquet(f'Files/data/{latest_total_count_file}')
fact_table = spark.read.parquet(f'Files/data/{latest_fact_file}')

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 19, Finished, Available, Finished)

data_total_20250423 085525.parquet
fact_table_20250423 083536.parquet


In [ ]:
display(total_count.show(5))
print('\n')
display(fact_table.show(5))

StatementMeta(, 581d04cb-7c65-4fb0-8af2-ab4ae3536c74, 10, Finished, Available, Finished)

+--------------------+--------------------+--------+----------+---------+
|           real_name|                name|userType|user_count|       id|
+--------------------+--------------------+--------+----------+---------+
|       VT 12 Messilä|               JK IN|       1|         0|101001874|
|       VT 12 Messilä|              JK OUT|       1|         0|102001874|
|       VT 12 Messilä|               PP IN|       2|         0|103001874|
|       VT 12 Messilä|              PP OUT|       2|         0|104001874|
|Hollolank (Mytäjä...|Hollolank (Mytäjä...|       2|         0|101002456|
+--------------------+--------------------+--------+----------+---------+
only showing top 5 rows





+---------+----------+-----------+---------+-----+-----------------+
|       id|       pvm|viikonpäivä|user_type|määrä|__index_level_0__|
+---------+----------+-----------+---------+-----+-----------------+
|101004858|2025-04-14|   maanatai|        1|  504|                0|
|101004858|2025-04-15|    tiistai|        1|  518|                0|
|101004858|2025-04-16|keskiviikko|        1|  547|                0|
|101004858|2025-04-17|    torstai|        1|  444|                0|
|101004858|2025-04-18|  perjantai|        1|  257|                0|
+---------+----------+-----------+---------+-----+-----------------+
only showing top 5 rows



In [ ]:
total_count_ordered = total_count.select('id', 'real_name', 'name', 'userType', 'user_count')
total_count_ordered.show(5)

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 30, Finished, Available, Finished)

+---------+--------------------+--------------------+--------+----------+
|       id|           real_name|                name|userType|user_count|
+---------+--------------------+--------------------+--------+----------+
|101001874|       VT 12 Messilä|               JK IN|       1|         0|
|102001874|       VT 12 Messilä|              JK OUT|       1|         0|
|103001874|       VT 12 Messilä|               PP IN|       2|         0|
|104001874|       VT 12 Messilä|              PP OUT|       2|         0|
|101002456|Hollolank (Mytäjä...|Hollolank (Mytäjä...|       2|         0|
+---------+--------------------+--------------------+--------+----------+
only showing top 5 rows



In [ ]:
total_count_ordered = total_count_ordered.drop('real_name')

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 31, Finished, Available, Finished)

In [ ]:
total_count_ordered = total_count_ordered.withColumnRenamed('id', 'asema_id')

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 32, Finished, Available, Finished)

In [ ]:
total_count_ordered.show(3)

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 33, Finished, Available, Finished)

+---------+------+--------+----------+
| asema_id|  name|userType|user_count|
+---------+------+--------+----------+
|101001874| JK IN|       1|         0|
|102001874|JK OUT|       1|         0|
|103001874| PP IN|       2|         0|
+---------+------+--------+----------+
only showing top 3 rows



In [ ]:
from pyspark.sql.functions import col, row_number
from pyspark.sql import Window

window_spec = Window.orderBy('asema_id')
total_count_ordered = total_count_ordered.withColumn('id', row_number().over(window_spec))
#total_count_ordered = total_count_ordered.select('id, asema_id', 'userType', 'user_count')
total_count_ordered.show(5)

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 34, Finished, Available, Finished)

+---------+--------------------+--------+----------+---+
| asema_id|                name|userType|user_count| id|
+---------+--------------------+--------+----------+---+
|101001874|               JK IN|       1|         0|  1|
|101002456|Hollolank (Mytäjä...|       2|         0|  2|
|101004858|      JK_Keskustaan2|       1|      2728|  3|
|101005848|       Keskustaan_IN|       1|      1112|  4|
|101013931|Uudenmaankatu/Rad...|       2|         0|  5|
+---------+--------------------+--------+----------+---+
only showing top 5 rows



In [ ]:
total_count_ordered = total_count_ordered.select('id', 'asema_id', 'userType', 'user_count')
total_count_ordered.show(3)

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 35, Finished, Available, Finished)

+---+---------+--------+----------+
| id| asema_id|userType|user_count|
+---+---------+--------+----------+
|  1|101001874|       1|         0|
|  2|101002456|       2|         0|
|  3|101004858|       1|      2728|
+---+---------+--------+----------+
only showing top 3 rows



In [ ]:
fact_table.show(5)

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 20, Finished, Available, Finished)

+---------+----------+-----------+---------+-----+-----------------+
|       id|       pvm|viikonpäivä|user_type|määrä|__index_level_0__|
+---------+----------+-----------+---------+-----+-----------------+
|101004858|2025-04-14|   maanatai|        1|  504|                0|
|101004858|2025-04-15|    tiistai|        1|  518|                0|
|101004858|2025-04-16|keskiviikko|        1|  547|                0|
|101004858|2025-04-17|    torstai|        1|  444|                0|
|101004858|2025-04-18|  perjantai|        1|  257|                0|
+---------+----------+-----------+---------+-----+-----------------+
only showing top 5 rows



In [ ]:
fact_table = fact_table.drop('__index_level_0__')
fact_table.show(3)

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 21, Finished, Available, Finished)

+---------+----------+-----------+---------+-----+
|       id|       pvm|viikonpäivä|user_type|määrä|
+---------+----------+-----------+---------+-----+
|101004858|2025-04-14|   maanatai|        1|  504|
|101004858|2025-04-15|    tiistai|        1|  518|
|101004858|2025-04-16|keskiviikko|        1|  547|
+---------+----------+-----------+---------+-----+
only showing top 3 rows



In [ ]:
fact_table = fact_table.withColumnRenamed('id', 'asema_id')
fact_table.show(3)

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 22, Finished, Available, Finished)

+---------+----------+-----------+---------+-----+
| asema_id|       pvm|viikonpäivä|user_type|määrä|
+---------+----------+-----------+---------+-----+
|101004858|2025-04-14|   maanatai|        1|  504|
|101004858|2025-04-15|    tiistai|        1|  518|
|101004858|2025-04-16|keskiviikko|        1|  547|
+---------+----------+-----------+---------+-----+
only showing top 3 rows



In [ ]:
from pyspark.sql.functions import col, row_number
from pyspark.sql import Window

window_spec = Window.orderBy('pvm', 'asema_id')
fact_table = fact_table.withColumn('id', row_number().over(window_spec))
fact_table.show(5)

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 23, Finished, Available, Finished)

+---------+----------+-----------+---------+-----+---+
| asema_id|       pvm|viikonpäivä|user_type|määrä| id|
+---------+----------+-----------+---------+-----+---+
|101004858|2025-04-14|   maanatai|        1|  504|  1|
|101005848|2025-04-14|   maanatai|        1|  203|  2|
|101034375|2025-04-14|   maanatai|        2|  890|  3|
|101059258|2025-04-14|   maanatai|        1| 2235|  4|
|101059446|2025-04-14|   maanatai|        1| 2707|  5|
+---------+----------+-----------+---------+-----+---+
only showing top 5 rows



In [ ]:
fact_table = fact_table.select('id', 'pvm', 'asema_id', 'viikonpäivä', 'määrä')

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 24, Finished, Available, Finished)

In [ ]:
fact_table.show(5)

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 25, Finished, Available, Finished)

+---+----------+---------+-----------+-----+
| id|       pvm| asema_id|viikonpäivä|määrä|
+---+----------+---------+-----------+-----+
|  1|2025-04-14|101004858|   maanatai|  504|
|  2|2025-04-14|101005848|   maanatai|  203|
|  3|2025-04-14|101034375|   maanatai|  890|
|  4|2025-04-14|101059258|   maanatai| 2235|
|  5|2025-04-14|101059446|   maanatai| 2707|
+---+----------+---------+-----------+-----+
only showing top 5 rows



In [ ]:
fact_table.printSchema()

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 26, Finished, Available, Finished)

root
 |-- id: integer (nullable = false)
 |-- pvm: string (nullable = true)
 |-- asema_id: long (nullable = true)
 |-- viikonpäivä: string (nullable = true)
 |-- määrä: long (nullable = true)



In [ ]:
fact_table = fact_table.withColumn('pvm', fact_table['pvm'].cast('date'))

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 27, Finished, Available, Finished)

In [ ]:
print(fact_table.count(), total_count.count())

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 28, Finished, Available, Finished)

295 56


In [ ]:
fact_table = fact_table.join(total_count_ordered, fact_table.asema_id == total_count_ordered.asema_id, 'left')\
.select(fact_table['*'], total_count_ordered['userType'])

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 36, Finished, Available, Finished)

In [ ]:
fact_table.count()

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 37, Finished, Available, Finished)

295

In [ ]:
fact_table.show(5)

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 38, Finished, Available, Finished)

+---+----------+---------+-----------+-----+--------+
| id|       pvm| asema_id|viikonpäivä|määrä|userType|
+---+----------+---------+-----------+-----+--------+
|  1|2025-04-14|101004858|   maanatai|  504|       1|
|  2|2025-04-14|101005848|   maanatai|  203|       1|
|  3|2025-04-14|101034375|   maanatai|  890|       2|
|  4|2025-04-14|101059258|   maanatai| 2235|       1|
|  5|2025-04-14|101059446|   maanatai| 2707|       1|
+---+----------+---------+-----------+-----+--------+
only showing top 5 rows



In [14]:
missing_vals = fact_table.filter(fact_table['userType'].isNull())
missing_vals.show()

StatementMeta(, 581d04cb-7c65-4fb0-8af2-ab4ae3536c74, 30, Finished, Available, Finished)

+---+---+--------+-----------+-----+--------+
| id|pvm|asema_id|viikonpäivä|määrä|userType|
+---+---+--------+-----------+-----+--------+
+---+---+--------+-----------+-----+--------+



## Taulujen päivittäminen

In [ ]:
print(total_count_ordered.count())
total_count_ordered.where('user_count > 0').show(10)

StatementMeta(, 581d04cb-7c65-4fb0-8af2-ab4ae3536c74, 37, Finished, Available, Finished)

56
+---+---------+--------+----------+
| id| asema_id|userType|user_count|
+---+---------+--------+----------+
|  3|101004858|       1|      2728|
|  4|101005848|       1|      1112|
|  6|101034375|       2|      5744|
|  7|101039385|       1|       437|
|  8|101059258|       1|     14671|
|  9|101059446|       1|     16199|
| 10|101061358|       1|      1533|
| 11|101061359|       1|       827|
| 13|101062896|       2|       881|
| 15|102004858|       1|      3267|
+---+---------+--------+----------+
only showing top 10 rows



In [ ]:
total_count_ordered.dtypes

StatementMeta(, 581d04cb-7c65-4fb0-8af2-ab4ae3536c74, 44, Finished, Available, Finished)

[('id', 'int'),
 ('asema_id', 'bigint'),
 ('userType', 'bigint'),
 ('user_count', 'bigint')]

## Using SQL to update total_count table
- Creating temporary view from source df using Python
- Updating delta table using SQL

In [ ]:
# Viikkodatasta väliaikainen näkymä
total_count_ordered.createOrReplaceTempView('source_totals_view')

StatementMeta(, 581d04cb-7c65-4fb0-8af2-ab4ae3536c74, 52, Finished, Available, Finished)

In [ ]:
%%sql
merge into LH_Lahti.dbo.total_count as target
using source_totals_view as source
on target.id = source.id
when MATCHED then
update set target.user_count = target.user_count + source.user_count

StatementMeta(, 581d04cb-7c65-4fb0-8af2-ab4ae3536c74, 57, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

## Using Python / Pyspark to update fact table

In [1]:
df = spark.sql("SELECT * FROM LH_Lahti.dbo.fact_table")
print(f'Taulussa on {df.count()} riviä ennen päivitystä.')

StatementMeta(, 2e65a519-0cee-44b3-96da-b641d0cbe07e, 3, Finished, Available, Finished)

Taulussa on 39526 riviä ennen päivitystä.


In [ ]:
print(f'Lisättäviä rivejä on {fact_table.count()}.')

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 39, Finished, Available, Finished)

Lisättäviä rivejä on 295.


In [ ]:
fact_table.write.format('delta').mode('append').save('abfss://Lahti@onelake.dfs.fabric.microsoft.com/LH_Lahti.Lakehouse/Tables/dbo/fact_table')

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 42, Finished, Available, Finished)

In [ ]:
df = spark.sql("SELECT * FROM LH_Lahti.dbo.fact_table")
print(f'Taulussa on {df.count()} riviä päivityksen jälkeen.')

StatementMeta(, e2e41b88-2a90-4bcc-a08b-69ee55263748, 43, Finished, Available, Finished)

Taulussa on 37120 riviä päivityksen jälkeen.
